# Running Documentation with Config

This notebook guides you through using the `run_documentation_tests()` function within the ValidMind Developer Framework. The function takes `config` as a parameter that enables developers to configure inputs and parameters for individual tests.

As a model developer, configuring individual tests is useful in various models development scenarios. For instance, based on a use case, a model might require changing inputs and/or parameters for certain tests. The `run_documentation_tests()` function allows you to directly configure tests through `config`, thus giving you flexibility to run tests according to your use case.

This guide includes the code required to:

- Load the demo dataset
- Prepocess the raw dataset
- Train a model for testing
- Initialize ValidMind objects
- Run documentation tests with custom configuration


## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:


In [ ]:
vm.preview_template()

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:


In [ ]:
# Import the sample dataset from the library

from validmind.datasets.classification import customer_churn as demo_dataset

print(
    f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}' \n\t• Class labels: {demo_dataset.class_labels}"
)

raw_df = demo_dataset.load_data()
raw_df.head()

## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.


## Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize XGBoost classifier: Creates an `XGBClassifier` object with early stopping rounds set to 10.
- Set evaluation metrics: Specifies metrics for model evaluation as "error," "logloss," and "auc."
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

## Train a model for testing

We train a simple customer churn model for our test.


In [ ]:
import xgboost
%matplotlib inline

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

xgb = xgboost.XGBClassifier(early_stopping_rounds=10)
xgb.set_params(
    eval_metric=["error", "logloss", "auc"],
)
xgb.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

## Initialize ValidMind objects


### Initialize ValidMind model object

Before you can run tests, you must first initialize a ValidMind model object using the [`init_model`](https://docs.validmind.ai/validmind/validmind.html#init_model) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `model` — the model that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test


In [ ]:
vm_model_xgb = vm.init_model(
    xgb,
    input_id="xgb",
)

### Initialize the ValidMind datasets

Similarly, initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset
- `class_labels` — an optional value to map predicted classes to class labels

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):


In [ ]:
vm_raw_ds = vm.init_dataset(
    input_id="raw_dataset",
    dataset=raw_df,
    target_column=demo_dataset.target_column,
)

feature_columns = [
    "CreditScore",
    "Gender",
    "Age",
    "Tenure",
    "Balance",
    "NumOfProducts",
    "HasCrCard",
    "IsActiveMember",
    "EstimatedSalary",
    "Geography_France",
    "Geography_Germany",
    "Geography_Spain",
]

vm_train_ds = vm.init_dataset(
    input_id="train_dataset",
    dataset=train_df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns,
)

vm_test_ds = vm.init_dataset(
    input_id="test_dataset",
    dataset=test_df,
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns,
)

### Run predictions through `assign_predictions` interface

We can use `assign_predictions()` to run and assign model predictions to our training and test datasets:


In [ ]:
vm_train_ds.assign_predictions(model=vm_model_xgb)
vm_test_ds.assign_predictions(model=vm_model_xgb)

## Run documentation tests with custom configuration


### Preview config

You can preview the default config for the documentation template using the `vm.get_test_suite().get_default_config()` interface.


In [ ]:
import json

project_test_suite = vm.get_test_suite()
config = project_test_suite.get_default_config()
print("Suite Config: \n", json.dumps(config, indent=2))

### Updating config

The test configuration can be updated to fit with your use case and requirements


In [ ]:
config = {
    "validmind.data_validation.DatasetSplit": {
        "inputs": {"datasets": (vm_train_ds, vm_test_ds)},
    },
    "validmind.model_validation.sklearn.PopulationStabilityIndex": {
        "inputs": {"model": vm_model_xgb, "datasets": (vm_train_ds, vm_test_ds)},
    },
    "validmind.model_validation.sklearn.ConfusionMatrix": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.ClassifierPerformance:in_sample": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_train_ds},
    },
    "validmind.model_validation.sklearn.ClassifierPerformance:out_of_sample": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.PrecisionRecallCurve": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.ROCCurve": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.TrainingTestDegradation": {
        "inputs": {"model": vm_model_xgb, "datasets": (vm_train_ds, vm_test_ds)},
    },
    "validmind.model_validation.sklearn.MinimumAccuracy": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.MinimumF1Score": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.MinimumROCAUCScore": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.PermutationFeatureImportance": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.SHAPGlobalImportance": {
        "inputs": {"model": vm_model_xgb, "dataset": vm_test_ds},
    },
    "validmind.model_validation.sklearn.WeakspotsDiagnosis": {
        "inputs": {"model": vm_model_xgb, "datasets": (vm_train_ds, vm_test_ds)},
    },
    "validmind.model_validation.sklearn.OverfitDiagnosis": {
        "inputs": {"model": vm_model_xgb, "datasets": (vm_train_ds, vm_test_ds)},
    },
    "validmind.model_validation.sklearn.RobustnessDiagnosis": {
        "inputs": {"model": vm_model_xgb, "datasets": (vm_train_ds, vm_test_ds)},
    },
}

### Run documentation tests

You can now run all documentation tests and pass an extra `config` parameter that overrides input and parameter configuration for the tests specified in the object.


In [ ]:
full_suite = vm.run_documentation_tests(
    inputs={
        "dataset": vm_raw_ds,
        "model": vm_model_xgb,
    },
    config=config,
)